# 1. Load Dataset

## Audio data
    - wav data와 이와 매칭되는 json label data를 model input이 가능하게 변환 필요
    - WhisperFeatureExtractor는 16kHz로 sampling된 데이터를 입력받는다


In [1]:
# set GPU
# torch 보다 먼저 import 해야 아래 사항이 적용된다
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "1"  # Set the GPU 1 to use
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


In [2]:
import glob
from datasets import load_dataset

# File path
training_file_dir = "/data/freetalk_senior/1.Training/"

training_raw_data = []
training_raw_data.append(sorted(glob.glob(os.path.join(training_file_dir, "raw_data/1.AI챗봇/1.AI챗봇_2_자유대화(노인남여)_TRAINING/노인남여_노인대화12_M_1534748575_63_강원_실내/*.wav"), recursive=True)))
training_raw_data.append(sorted(glob.glob(os.path.join(training_file_dir, "raw_data/1.AI챗봇/1.AI챗봇_2_자유대화(노인남여)_TRAINING/노인남여_노인대화12_M_1539661748_64_수도권_실내/*.wav"), recursive=True)))
training_raw_data.append(sorted(glob.glob(os.path.join(training_file_dir, "raw_data/1.AI챗봇/1.AI챗봇_2_자유대화(노인남여)_TRAINING/노인남여_노인대화12_M_1539661748_64_수도권_실외/*.wav"), recursive=True)))

training_labeled_data = []
training_labeled_data.append(sorted(glob.glob(os.path.join(training_file_dir, "labeled_data/1.AI챗봇/1.AI챗봇_라벨링_자유대화(노인남여)_TRAINING/노인남여_노인대화12_M_1534748575_63_강원_실내/*.json"), recursive=True)))
training_labeled_data.append(sorted(glob.glob(os.path.join(training_file_dir, "labeled_data/1.AI챗봇/1.AI챗봇_라벨링_자유대화(노인남여)_TRAINING/노인남여_노인대화12_M_1539661748_64_수도권_실내/*.json"), recursive=True)))
training_labeled_data.append(sorted(glob.glob(os.path.join(training_file_dir, "labeled_data/1.AI챗봇/1.AI챗봇_라벨링_자유대화(노인남여)_TRAINING/노인남여_노인대화12_M_1539661748_64_수도권_실외/*.json"), recursive=True)))


In [3]:
len(training_raw_data[0] + training_raw_data[1] + training_raw_data[2])

2086

In [4]:
len(training_labeled_data[0]+ training_labeled_data[1] + training_labeled_data[2])

2086

In [5]:
data_files = {
    "train_input": training_raw_data,    
    "train_label": training_labeled_data,
}

In [6]:

# Load Dataset
from datasets import Features, Audio
import datasets 

# load_dataset 이 새로 만들어 버리는 label column 때문에 에러 발생하여 features명시
features = Features({
    'audio': Audio()
})


train_input_0 = load_dataset("audiofolder", 
                           data_files=data_files['train_input'][0], 
                           features=features)
train_input_1 = load_dataset("audiofolder", 
                           data_files=data_files['train_input'][1], 
                           features=features)
train_input_2 = load_dataset("audiofolder", 
                           data_files=data_files['train_input'][2], 
                           features=features)

train_input = datasets.DatasetDict({
    "train": datasets.concatenate_datasets([train_input_0['train'], 
                                            train_input_1['train'], 
                                            train_input_2['train']], axis=0)
})

train_input

Resolving data files:   0%|          | 0/500 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1554 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['audio'],
        num_rows: 2086
    })
})

In [7]:
import datasets 

train_label_0 = load_dataset("json", data_files=data_files["train_label"][0])
train_label_1 = load_dataset("json", data_files=data_files["train_label"][1])
train_label_2 = load_dataset("json", data_files=data_files["train_label"][2])

train_label = datasets.DatasetDict({
    "train": datasets.concatenate_datasets([train_label_0['train'], 
                                            train_label_1['train'], 
                                            train_label_2['train']], axis=0)
})

train_label

Resolving data files:   0%|          | 0/500 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1554 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['발화정보', '대화정보', '녹음자정보'],
        num_rows: 2086
    })
})

In [8]:
train_dataset = datasets.DatasetDict({
    "train": datasets.concatenate_datasets([train_input['train'], 
                                            train_label['train']], axis=1)
})
train_dataset

DatasetDict({
    train: Dataset({
        features: ['audio', '발화정보', '대화정보', '녹음자정보'],
        num_rows: 2086
    })
})

In [9]:
senior_dataset = train_dataset.shuffle(seed=44)

In [10]:
senior_dataset.column_names["train"][2:4]

['대화정보', '녹음자정보']

# 2. Set Toknizer & FeatureExtractor

## Feature Extractor: Audio raw data에 대 preprocessing 한다
    - Transformers.WhisperFeatureExtractor
    1. Audio sample에 대해 padding 및 truncating을 해 모든 sample길이를 30초 맞춘다.
        whisper model은 attention mask없이 발화 신호로부터 무시해야 할 input을 직접 추론하도록 학습되었다
    2. Audio array를 log-Mel spectrum으로 변환. -> 이 것이 model input

## Model: seq-to-seq mapping

## Tokeninzer: model output을 text format으로 postprocessing
    - Transformers.WhisperTokenizer


In [17]:
checkpoint = "jiwon65/whisper-small_korean-zeroth"

# Whisper model output은 vocabulary 단어의 index
# 이를 실제 문자와 mapping하기 위해 Tokenizer 사용

from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained(checkpoint, language="Korean", task="transcribe")
# from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained(checkpoint, language="Korean", task="transcribe")

# Feature Extractor
from transformers import WhisperFeatureExtractor

# jiwon65/whisper-small_korean-zeroth model을 fine tuning
feature_extractor = WhisperFeatureExtractor.from_pretrained(checkpoint)
# feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [18]:
# WhisperProcessor: 위 두 모듈을 하나로 묶는다
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(checkpoint, language="Korean", task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# 3. Dataset Preprocessing

In [13]:
def preprocess_dataset(batch):
    audio=batch["audio"]
    label=batch["발화정보"]
    
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    batch["labels"] = tokenizer(label['stt']).input_ids
    return batch

senior_dataset = senior_dataset.map(preprocess_dataset, 
                                    remove_columns=senior_dataset.column_names["train"], 
                                    # batched=True,
                                    # batch_size=1,
                                    # num_proc=os.cpu_count()
                                   )

In [14]:
senior_dataset = senior_dataset.shuffle(seed=44)
splited_senior_dataset = senior_dataset["train"].train_test_split(test_size=0.2)

In [15]:
splited_senior_dataset

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 1668
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 418
    })
})

# 4. Set Trainer & Training

## 4-1. Set DataCollator

    seq-to-seq model을 위한 DataCollator는 input_feature와 label을 독립적으로 다룬다는 점에서 특징적.
        input_feature -> FeatureExtractor
        label -> Tokenizer 로 다뤄야 함

In [16]:
# input_feature -> log_mel_spectrogram -> PyTorch Tensor
import torch
from dataclasses import dataclass
from typing import Any, Dict, List, Union

# DatasetDict -> torch.tensor
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # 인풋 데이터와 라벨 데이터의 길이가 다르며, 따라서 서로 다른 패딩 방법이 적용되어야 한다. 그러므로 두 데이터를 분리해야 한다.
        # 먼저 오디오 인풋 데이터를 간단히 토치 텐서로 반환하는 작업을 수행한다.
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")
        
        # Tokenize된 레이블 시퀀스를 가져온다.
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # 레이블 시퀀스에 대해 최대 길이만큼 패딩 작업을 실시한다.
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")
        
        # 패딩 토큰을 -100으로 치환하여 loss 계산 과정에서 무시되도록 한다.
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        
        # 이전 토크나이즈 과정에서 bos 토큰이 추가되었다면 bos 토큰을 잘라낸다.
        # 해당 토큰은 이후 언제든 추가할 수 있다.
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]
        
        batch["labels"] = labels
        
        return batch

In [17]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(checkpoint, language="Korean", task="transcribe")

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## 4-2. Set metric

In [18]:
import evaluate

metric = evaluate.load('cer')

In [19]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    label_ids[label_ids == -11] = tokenizer.pad_token_id

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    cer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"cer": cer}

## 4-3. Set Train args

    output_dir: model weight를 저장하는 경로
    generation_max_length: evaluation동안 recursive하게 생성되는 token들의 최대 길이 
    save_steps: Training동안, 이 parameter에 설정한 step 마다 checkpoint 저장
    eval_steps: Training동안, 이 parameter에 설정한 step 마다 checkpoint 평가 수행
    report_to: training_log를 어디 저장할 지 설정. default: tensorboard

In [ ]:
# pretrained_model load
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained(checkpoint)

In [21]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [22]:
from transformers import Seq2SeqTrainingArguments
steps = 500
training_args = Seq2SeqTrainingArguments(
    output_dir="./test_trainer",       # output dir
    # logging_dir="./logging_dir",       # storing to log_dir
    max_steps=steps,                   # epoch 대신 설정
    # num_train_epochs = 5,              # total num of training epochs
    evaluation_strategy="steps",       # evaluate after each "steps"

    warmup_steps=steps/20,             # num of warmup steps for learning rate scheduler
    save_steps=steps/10,
    eval_steps=steps/20,
    logging_steps=steps/50,            # how often to print log
    
    per_device_train_batch_size=16,    # batch size of per_device during 
    per_device_eval_batch_size=16,      # batch size per GPU for evaluation
    # per_device_eval_batch_size=10,   # batch size for evaluation
    gradient_accumulation_steps=2,     # tot num of steps before back propagation. train time increase.
    learning_rate=1e-5,                # initial lr for AdamW optimizer
    # weight_decay=0.01,                 # strength of weight decay
    gradient_checkpointing=True,       # saving mem, but make backward pass slower
    fp16=True,                         # use mixed precision
    # fp16_opt_level="02",               # mixed precision mode
    
    # predict_with_generate=True,        # using generate to calculate 'generative metrics'(ROUGE, BLEU)
    # generation_max_length=225,         # generative metrics's max len

    report_to=["tensorboard"],
    load_best_model_at_end=True,       # best checkpoint will always be saved
    
    metric_for_best_model="cer",       # 한국어의 경우 띄어쓰기가 애매한 경우가 많아서 'wer'보다는 'cer'이 더 적합할 것
    greater_is_better=False,           # is better metric result is big?
    push_to_hub=False,
    # do_train=True,                   # Perform training
    # do_eval=True,                    # Perform evaluation
    run_name="test1800Training",       # experiment name
    # seed=3,                            # seed for experiment reproductbility 3x3
)

/home/wooseok/anaconda3/envs/whisper_kor/lib/python3.9/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## 4-4. Set Trainer

In [23]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=splited_senior_dataset["train"],
    eval_dataset=splited_senior_dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

max_steps is given, it will override any value given in num_train_epochs


## 4-5. Set GPU

In [24]:
# 맨 처음 set

In [2]:
print("CUDA_DEVICE_ORDER:", os.environ.get("CUDA_DEVICE_ORDER"))
print("CUDA_VISIBLE_DEVICES:", os.environ.get("CUDA_VISIBLE_DEVICES"))

CUDA_DEVICE_ORDER: PCI_BUS_ID
CUDA_VISIBLE_DEVICES: 1


In [ ]:
# torch 를 먼저 import 하면 CUDA_VISIBLE_DEVICES 설정이 안먹음
import torch
print("Number of GPUs available:", torch.cuda.device_count())

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

model.config.use_cache = False # mem usage 줄이기 위함

In [27]:
print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

Device: cuda:0
Current cuda device: 0
Count of using GPUs: 1


In [ ]:
torch.cuda.empty_cache()

trainer.train() # data 1800 개

In [6]:
model_tuned = WhisperForConditionalGeneration.from_pretrained("./test_trainer/checkpoint-150/")

In [ ]:
trainer2 = Seq2SeqTrainer(
    args=training_args,
    model=model_tuned,
    train_dataset=splited_senior_dataset["train"],
    eval_dataset=splited_senior_dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

In [36]:
trainer2.evaluate()

{'eval_loss': 0.15303942561149597,
 'eval_cer': 7.777777777777778,
 'eval_runtime': 106.0101,
 'eval_samples_per_second': 3.943,
 'eval_steps_per_second': 0.5}

In [7]:
model_raw = WhisperForConditionalGeneration.from_pretrained("jiwon65/whisper-small_korean-zeroth")

In [40]:
trainer.model = model_raw.to(device)

In [41]:
trainer.evaluate()

{'eval_loss': 1.1474889516830444, 'eval_cer': 29.860017497812773}